# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 計數編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [72]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '/content/drive/MyDrive/資料科學/第四屆基機器學習馬拉松/D30/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [73]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [31]:
# 觀察欄位相異值數量
df.select_dtypes(include=['object']).apply(pd.Series.nunique)

Name        1307
Sex            2
Ticket       929
Cabin        187
Embarked       4
dtype: int64

# 作業2
* 承上題，三者比較效果何者最好?

In [33]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression(max_iter=1000)
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7800138095537004


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [37]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(by=['Cabin']).count()['Name'].reset_index()
# 由於計算完Cabin個基數的數量後，其欄位名稱並不正確，在此更動
count_df = count_df.rename(columns={'Name':'Cabin_Count'})
df_temp = pd.merge(df, count_df, on=['Cabin'], how='left')
count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)

,Cabin,Cabin_Count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [38]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
# 將其他類別特徵轉換
for c in object_features:
  df_temp[c] = LabelEncoder().fit_transform(df_temp[c])

train_X = df_temp[:train_num]
print('Count_邏輯回歸_score:{}'.format(cross_val_score(LogisticRegression(max_iter=1000), train_X, train_Y, cv=5).mean()))


Count_邏輯回歸_score:0.7822547234950725


In [64]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_temp = pd.DataFrame()
# 先將類別型特徵用標籤編碼轉換
for c in object_features:
  df_temp[c] = LabelEncoder().fit_transform(df[c])
# 直接將'Cabin'中的名稱放入hash()中得取輸出數值，為了要確定是緊密(dense)特徵, 因此除以10後看餘數
# 意思是將數值範圍限制在10之內(0~9)
df_temp['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10).reset_index()['Cabin']
train_X = df_temp[:train_num]
print('hash_邏輯斯回歸_score:{}'.format(cross_val_score(LogisticRegression(max_iter=1000), train_X, train_Y, cv=5).mean()))
df_temp.head()

hash_邏輯斯回歸_score:0.7867365513778168


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,0
1,286,0,816,106,0,2
2,523,0,914,185,3,0
3,422,0,65,70,3,0
4,22,1,649,185,3,0


In [77]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸

# 計數編碼

count_df = df.groupby(by=['Cabin']).count()['Name'].reset_index()
count_df = count_df.rename(columns={'Name':'Cabin_Count'})
df_temp = pd.merge(df, count_df, on=['Cabin'], how='left')

# Cabin 特徵雜湊
df_temp['Cabin_Hash'] = df_temp['Cabin'].map(lambda x:hash(x)%10).reset_index()['Cabin']

# 其他特徵轉換

for c in object_features:
  df_temp[c] = LabelEncoder().fit_transform(df_temp[c])

train_X = df_temp[:train_num]
# 建模

print('Cabin_Count_Hash_邏輯斯回歸_score:{}'.format(cross_val_score(LogisticRegression(max_iter=1000), train_X, train_Y, cv=5).mean()))

Cabin_Count_Hash_邏輯斯回歸_score:0.7800012554139727


In [78]:
train_X.head()

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count,Cabin_Hash
0,155,1,720,185,3,1014,0
1,286,0,816,106,0,2,2
2,523,0,914,185,3,1014,0
3,422,0,65,70,3,2,0
4,22,1,649,185,3,1014,0
